## Geração Automática de Texto com LSTMs

As Redes Neurais Recorrentes também funcionam como modelos generativos. Além de prever, elas aprendem sequências de um problema e podem criar novas sequências plausíveis no mesmo domínio. Esses modelos generativos são valiosos não só para avaliar o aprendizado de um problema, mas também para entender melhor o domínio em questão.

Para aprimorar a habilidade de escrita, é útil usar livros clássicos, onde já temos familiaridade com a história e não violamos direitos autorais. Muitos livros clássicos estão em domínio público e podem ser acessados gratuitamente. O Projeto Gutenberg é uma ótima fonte para encontrar esses livros. Utilizaremos "Alice no País das Maravilhas" ou "Alice's Adventures in Wonderland" em inglês. O arquivo txt do livro está disponível em https://www.gutenberg.org/ebooks/11 ou anexado a este Jupyter Notebook, contendo cerca de 3.300 linhas de texto, sem o cabeçalho e a marca de final de arquivo.

Exploraremos as relações entre caracteres e suas probabilidades condicionais em sequências para gerar novas sequências de caracteres originais. É uma atividade divertida e sugiro experimentar com outros livros do Projeto Gutenberg. Além de texto, é possível usar outros dados ASCII, como código de programação, documentos em LaTeX, HTML ou Markdown, entre outros.

Nossa abordagem será similar à do programador destacada neste artigo: http://www.businessinsider.com/ai-just-wrote-the-next-book-of-game-of-thrones-for-us-2017-8

In [1]:
!git clone https://github.com/FIAPON/fiap-deep-learning.git

Cloning into 'fiap-deep-learning'...
remote: Enumerating objects: 8267, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 8267 (delta 48), reused 46 (delta 15), pack-reused 8163
Receiving objects: 100% (8267/8267), 533.65 MiB | 22.89 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Updating files: 100% (8238/8238), done.


In [2]:
%cd /content/fiap-deep-learning/RNNS

/content/fiap-deep-learning/RNNS


In [3]:
# Imports
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

In [4]:
# Carregamos os dados e convertemos para lowercase
filename = "dataset/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

Agora que o livro está carregado, devemos preparar os dados para modelagem. Não podemos modelar os caracteres diretamente, em vez disso, devemos converter os caracteres em números inteiros. Podemos fazer isso facilmente, criando um conjunto de todos os caracteres distintos do livro, então criando um mapa de cada caractere para um único inteiro.

In [5]:
# Criando o mapeamento caracter/inteiro
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [6]:
chars

['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 ':',
 ';',
 '?',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\ufeff']

In [7]:
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 '*': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 ':': 11,
 ';': 12,
 '?': 13,
 '[': 14,
 ']': 15,
 '_': 16,
 'a': 17,
 'b': 18,
 'c': 19,
 'd': 20,
 'e': 21,
 'f': 22,
 'g': 23,
 'h': 24,
 'i': 25,
 'j': 26,
 'k': 27,
 'l': 28,
 'm': 29,
 'n': 30,
 'o': 31,
 'p': 32,
 'q': 33,
 'r': 34,
 's': 35,
 't': 36,
 'u': 37,
 'v': 38,
 'w': 39,
 'x': 40,
 'y': 41,
 'z': 42,
 '\ufeff': 43}

Pode haver alguns caracteres que podemos remover para limpar mais o conjunto de dados que reduzirá o vocabulário e poderá melhorar o processo de modelagem.

In [8]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  144343
Total Vocab:  44


Podemos ver que o livro tem pouco menos de 150.000 caracteres e que quando convertidos para minúsculas, existem apenas 44 caracteres distintos no vocabulário para a rede aprender, muito mais do que os 26 no alfabeto. Agora, precisamos definir os dados de treinamento para a rede. Existe muita flexibilidade em como você escolhe dividir o texto e expô-lo a rede durante o treino. Aqui dividiremos o texto do livro em subsequências com um comprimento de 100 caracteres, um comprimento arbitrário. Poderíamos facilmente dividir os dados por sentenças e ajustar as sequências mais curtas e truncar as mais longas. Cada padrão de treinamento da rede é composto de 100 passos de tempo (time steps) de um caractere (X) seguido por um caracter de saída (y). Ao criar essas sequências, deslizamos esta janela ao longo de todo o livro um caracter de cada vez, permitindo que cada caracter tenha a chance de ser aprendido a partir dos 100 caracteres que o precederam (exceto os primeiros 100 caracteres, é claro). Por exemplo, se o comprimento da sequência é 5 (para simplificar), os dois primeiros padrões de treinamento seriam os seguintes:

* Palavra: CHAPTER
* CHAPT -> E
* HAPTE -> R

In [9]:
# À medida que dividimos o livro em sequências, convertemos os caracteres em números inteiros usando nossa
# tabela de pesquisa que preparamos anteriormente.
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total de Padrões: ", n_patterns)

Total de Padrões:  144243


Agora que preparamos nossos dados de treinamento, precisamos transformá-lo para que possamos usá-lo com o Keras. Primeiro, devemos transformar a lista de sequências de entrada na forma [amostras, passos de tempo, recursos] esperados por uma rede LSTM. Em seguida, precisamos redimensionar os números inteiros para o intervalo de 0 a 1 para tornar os padrões mais fáceis de aprender pela rede LSTM que usa a função de ativação sigmoide por padrão.

In [10]:
# Reshape de X para [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# Normalização
X = X / float(n_vocab)

Finalmente, precisamos converter os padrões de saída (caracteres únicos convertidos em números inteiros) usando Hot-Encoding. Isto é para que possamos configurar a rede para prever a probabilidade de cada um dos 44 caracteres diferentes no vocabulário (uma representação mais fácil) em vez de tentar forçá-lo a prever com precisão o próximo caracter. Cada valor de y é convertido em um vetor com um comprimento 44, cheio de zeros, exceto com um 1 na coluna para a letra (inteiro) que o padrão representa. Por exemplo, quando a letra n (valor inteiro 30) tiver sido transformada usando One-Hot Encoding, vai se parecer com isso:

[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [11]:
# One-Hot Encoding da variável de saída
y = to_categorical(dataY)

In [12]:
# Modelo LSTM com duas camadas de Dropout com 20%
# O tempo de treinamento é bem longo
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Não há conjunto de dados de teste. Estamos modelando todo o conjunto de dados de treinamento para aprender a probabilidade de cada caracter em uma sequência. Não estamos interessados nos mais preciso modelo do conjunto de dados de treinamento (Acurácia de Classificação). Este seria um modelo que prevê cada caracter no conjunto de dados de treinamento perfeitamente. Em vez disso, estamos interessados em uma generalização do conjunto de dados que minimiza a função de perda escolhida. Estamos buscando um equilíbrio entre generalização e
overfitting.

In [13]:
# Define o checkpoint
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')
callbacks_list = [checkpoint]

Fit do modelo

In [14]:
# model.fit(X, y, epochs = 20, batch_size = 128, callbacks = callbacks_list)
model.fit(X, y, epochs = 50, batch_size = 64, callbacks = callbacks_list)

Epoch 1/50
2254/2254 [==============================] - ETA: 0s - loss: 2.7736
Epoch 1: loss improved from inf to 2.77360, saving model to weights-improvement-01-2.7736.hdf5
2254/2254 [==============================] - 46s 18ms/step - loss: 2.7736
Epoch 2/50
   7/2254 [..............................] - ETA: 39s - loss: 2.5264

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2253/2254 [============================>.] - ETA: 0s - loss: 2.3935
Epoch 2: loss improved from 2.77360 to 2.39349, saving model to weights-improvement-02-2.3935.hdf5
2254/2254 [==============================] - 42s 19ms/step - loss: 2.3935
Epoch 3/50
2254/2254 [==============================] - ETA: 0s - loss: 2.1911
Epoch 3: loss improved from 2.39349 to 2.19110, saving model to weights-improvement-03-2.1911.hdf5
2254/2254 [==============================] - 42s 19ms/step - loss: 2.1911
Epoch 4/50
2252/2254 [============================>.] - ETA: 0s - loss: 2.0591
Epoch 4: loss improved from 2.19110 to 2.05900, saving model to weights-improvement-04-2.0590.hdf5
2254/2254 [==============================] - 43s 19ms/step - loss: 2.0590
Epoch 5/50
2253/2254 [============================>.] - ETA: 0s - loss: 1.9624
Epoch 5: loss improved from 2.05900 to 1.96239, saving model to weights-improvement-05-1.9624.hdf5
2254/2254 [==============================] - 42s 19ms/step - loss: 1.9624
Epo

Depois de executar o fit, você deve ter uma série de arquivos de checkpoint no mesmo diretório onde está este Jupyter Notebook. Você pode excluí-los todos exceto aquele com o menor valor de perda. Por exemplo, neste caso, o arquivo weights-improvement-19-1.9119.hdf5 será usado. Ele contém os melhores valores de peso.

In [16]:
# Carrega os melhores pesos da rede e compila o modelo
filename = "/content/fiap-deep-learning/RNNS/weights-improvement-50-1.1944.hdf5"
model.load_weights(filename)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [17]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
# Obtém um random seed
start = numpy.random.randint(0, len(dataX)-1)

# Inicia a geração de texto de um ponto qualquer, definido pelo random seed "start"
pattern = dataX[start]
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# Gerando caracteres
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nConcluído.")

" d it out, we should all have our heads cut off, you know.
so you see, miss, we're doing our best, af "
iore the tame words ' 
'i'm a pittle shing i don't know what the rame thing!' said the dormouse, who was see anything to see it tailing to the wood, who was seeding to herself, 'i dare was the tood, and was in one fres to say '

'i con't know what this you?' said the dormouse, 'or i'd nade of the sood of the words ' (and the mock turtle said this she was suill in a mong wa

Abaixo algumas sugestões para melhorar este modelo:

* Prever menos de 1.000 caracteres como saída para uma determinada semente (seed).
* Remova toda a pontuação do texto original e, portanto, do vocabulário do modelo.
* Experimente One-Hot Encoding para as sequências de entrada.
* Aumente o número de épocas de treinamento para 100 ou muitas centenas (isso pode levar até dias para o treinamento, mas aumentará a previsão do modelo.
* Ajuste o percentual de Dropout
* Adicione mais unidades de memória às camadas e/ou mais camadas.